# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [5]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [6]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [7]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [8]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [21]:
def foldl2(f, x0, lst):
    return foldr(lambda a, b : (lambda x: b(f(x, a))), lambda x: x, lst)(x0)

In [22]:
def foldr2(f, x0, lst):
    return foldl(lambda a, b: lambda n: a(f(b, n)), lambda x: x, lst)(x0)

In [23]:
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [27]:
from string import ascii_letters

In [29]:
def check_inv(a, b):
    assert len(a) >= len(b)
    
    b_chars = dict.fromkeys(ascii_letters, 0)
    for char in b:
        b_chars[char] += 1
            
    curr_chars = dict.fromkeys(ascii_letters, 0)
    for char in a[:len(b) - 1]:
        curr_chars[char] += 1
            
    for i in range(len(b) - 1, len(a)):
        curr_chars[a[i]] += 1
        if(curr_chars == b_chars):
            return True
        curr_chars[a[i - len(b) + 1]] -= 1
        
    return False

In [31]:
check_inv( 'abcrotm', 'tro')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [33]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if(self.left == None and self.right == None):
            yield self.value
            
        if(self.left):
            yield from self.left
            
        if(self.right):
            yield from self.right
        
    
    def __str__(self):
        res = [str(self.value)]
        
        if(self.left):
            res.append(self.left.__str__())
            
        if(self.right):
            res.append(self.right.__str__())
            
        return ' '.join(res)
    
    def __repr__(self):
        if(self.left == None and self.right == None):
            return f"Tree({self.value})"
        
        if(self.right == None):
            return f"Tree({self.value}, {self.left.__repr__()})"
        
        if(self.left == None):
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [34]:
operations = {'+': (1, lambda x, y: x + y),
             '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y),
             '/': (2, lambda x, y: x / y)}

def parse(str_expr):
    num = ''
    res = []
    
    for s in str_expr:
        if s in '1234567890':
            num += s
            
        elif num:
            res.append(float(num))
            num = ''
        
        if s in '+-/*' or s in '()':
            res.append(s)
            
    if num:
        res.append(num)
        
    return res
def pol_notation(parsed_expr):
    res = []
    stack = []
    for token in parsed_expr:
        if token in operations:
            while stack and stack[-1] != '(' and operations[token][0] <= operations[stack[-1]][0]:
                res.append(stack.pop())
                
            stack.append(token)
            
        elif token == ')':
            while stack:
                x = stack.pop()
                
                if x == '(':
                    break
                    
                res.append(x)
                
        elif token == '(':
            stack.append(token)
            
        else:
            res.append(token)
            
    while stack:
        res.append(stack.pop())
        
    return res

def calc(expr):
    
    polish = pol_notation(parse(expr))
    
    stack = []
    for token in polish:
        if token not in operations:
            stack.append(float(token))
        if token in operations:
            y, x = stack.pop(), stack.pop()
            stack.append(operations[token][1](x, y))
            
    return stack.pop()

calc('2 * (15 - 3 * 4) - 2') == 4

True